# Age 


# Import Libraries, Dependencies and Dataset


In [7]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
import os

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_log_error, explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.stats import norm

from pytorch_tabnet.tab_model import TabNetRegressor
import torch

In [8]:
# Dataset download
path = kagglehub.dataset_download("imoore/age-dataset")

print("Path to dataset files:", path)
dataset_path = "/home/codespace/.cache/kagglehub/datasets/imoore/age-dataset/versions/1"
print("Files in dataset directory:", os.listdir(dataset_path))

file_path = os.path.join(dataset_path, "AgeDataset-V1.csv")  #
df = pd.read_csv(file_path)

print(df.head())
print(df.info())
print(df.describe())

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/imoore/age-dataset/versions/1
Files in dataset directory: ['AgeDataset-V1.csv']
     Id                     Name  \
0   Q23        George Washington   
1   Q42            Douglas Adams   
2   Q91          Abraham Lincoln   
3  Q254  Wolfgang Amadeus Mozart   
4  Q255     Ludwig van Beethoven   

                                 Short description Gender  \
0   1st president of the United States (1732–1799)   Male   
1                      English writer and humorist   Male   
2  16th president of the United States (1809-1865)   Male   
3        Austrian composer of the Classical period   Male   
4           German classical and romantic composer   Male   

                                             Country  Occupation  Birth year  \
0  United States of America; Kingdom of Great Bri...  Politician        1732   
1                                     United Kingdom      Artist        1952   
2                           Uni

# Preprocessing

In [13]:
# Initial Data Info
print(f"Initial dataset shape: {df.shape}")

# Step 1: Handle missing data
df['Gender'] = df['Gender'].fillna('Unknown')
df['Country'] = df['Country'].fillna('Unknown')
df['Occupation'] = df['Occupation'].fillna('Unknown')
df = df.dropna(subset=['Death year', 'Age of death'])
print(f"After dropping rows with missing targets: {df.shape}")

# Step 2: Drop 'Manner of death'
df = df.drop(columns=['Manner of death'], errors='ignore')
print(f"Dataset shape after dropping columns: {df.shape}")

# Step 3: Encode categorical variables (preserve as categorical for TabNet)
print("Encoding categorical variables...")
label_encoder = LabelEncoder()
df['Country'] = label_encoder.fit_transform(df['Country'])  # Keep as categorical index
df['Occupation'] = label_encoder.fit_transform(df['Occupation'])  # Keep as categorical index

# Step 4: Define features and target
target = 'Age of death'
X = df.drop(columns=[target, 'Id', 'Name'])  # Drop ID/name early
y = df[target]

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# Step 6: Process text features using TF-IDF (full dataset)
print("Processing text features...")
tfidf = TfidfVectorizer(max_features=50)  # Reduced features for efficiency
X_train_text = tfidf.fit_transform(X_train['Short description'].fillna(''))
X_test_text = tfidf.transform(X_test['Short description'].fillna(''))

# Convert to DataFrames and merge
text_features = [f"tfidf_{i}" for i in range(tfidf.max_features)]
X_train = pd.concat([
    X_train.reset_index(drop=True).drop(columns=['Short description']),
    pd.DataFrame(X_train_text.toarray(), columns=text_features)
], axis=1)

X_test = pd.concat([
    X_test.reset_index(drop=True).drop(columns=['Short description']),
    pd.DataFrame(X_test_text.toarray(), columns=text_features)
], axis=1)

# Step 7: One-hot encode gender
X_train = pd.get_dummies(X_train, columns=['Gender'], prefix='gender')
X_test = pd.get_dummies(X_test, columns=['Gender'], prefix='gender')

# Align columns between train/test
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Step 8: Scale numerical features
print("Scaling numerical features...")
numerical_cols = ['Birth year', 'Death year']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Step 9: Prepare categorical indices for TabNet (BEFORE converting to numpy)
categorical_columns = ['Country', 'Occupation']
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_columns]
categorical_dims = {
    X_train.columns.get_loc(col): len(df[col].unique())
    for col in categorical_columns
}

# Step 10: Convert to numpy arrays (AFTER getting categorical indices)
X_train = X_train.astype(np.float32).values
X_test = X_test.astype(np.float32).values
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

print("\nFinal Preprocessed Shapes:")
print(f"Train features: {X_train.shape}, Test features: {X_test.shape}")
print(f"Categorical indices: {categorical_indices}")
print(f"Categorical dimensions: {categorical_dims}")


# }

Initial dataset shape: (1223008, 9)
After dropping rows with missing targets: (1223008, 9)
Dataset shape after dropping columns: (1223008, 9)
Encoding categorical variables...
Train shape: (978406, 6), Test shape: (244602, 6)
Processing text features...
Scaling numerical features...

Final Preprocessed Shapes:
Train features: (978406, 74), Test features: (244602, 74)
Categorical indices: [0, 1]
Categorical dimensions: {0: 5962, 1: 9314}


# TabNet Modeling


In [14]:
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
import torch

# Convert data to numpy arrays
y_train = y_train.astype(np.float32).reshape(-1, 1)
y_test = y_test.astype(np.float32).reshape(-1, 1)

# Configure TabNet with categorical information
tabnet_params = {
    "cat_idxs": categorical_indices,
    "cat_dims": [categorical_dims[idx] for idx in categorical_indices],
    "cat_emb_dim": 2,  # Embedding dimension for categorical features
    "n_d": 64,  # Dimension of prediction layer
    "n_a": 64,  # Dimension of attention layer
    "n_steps": 5,  # Number of sequential attention steps
    "gamma": 1.3,  # Coefficient for feature reusage
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=1e-3, weight_decay=1e-5),
    "scheduler_params": {"step_size": 50, "gamma": 0.9},
    "scheduler_fn": torch.optim.lr_scheduler.StepLR,
    "mask_type": "entmax",  # Sparsemax or entmax for feature selection
    "verbose": 1,
}

# Initialize model
tabnet_model = TabNetRegressor(**tabnet_params)

# Train with early stopping
tabnet_model.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=["train", "valid"],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=30,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=4,
    drop_last=False,
)

# Plot training metrics
plt.figure(figsize=(10, 6))
plt.plot(tabnet_model.history["loss"], label="Training Loss")
plt.plot(tabnet_model.history["valid_rmse"], label="Validation RMSE")
plt.title("Training Metrics")
plt.xlabel("Epochs")
plt.ylabel("RMSE/Loss")
plt.legend()
plt.grid()
plt.show()

# Evaluate
y_pred = tabnet_model.predict(X_test).flatten()
print(f"Final RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or

epoch 0  | loss: 823.358 | train_rmse: 5.47972 | valid_rmse: 7.5928  |  0:03:32s


/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch 1  | loss: 8.02734 | train_rmse: 6.48606 | valid_rmse: 7.48127 |  0:07:03s


/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

epoch 58 | loss: 1.88752 | train_rmse: 0.71059 | valid_rmse: 0.71072 |  3:24:56s


/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch 59 | loss: 2.05889 | train_rmse: 1.73666 | valid_rmse: 1.74057 |  3:28:23s

Early stopping occurred at epoch 59 with best_epoch = 29 and best_valid_rmse = 0.53291


/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


: 

epoch 59 | loss: 2.05889 | train_rmse: 1.73666 | valid_rmse: 1.74057 |  3:28:23s

Early stopping occurred at epoch 59 with best_epoch = 29 and best_valid_rmse = 0.53291